### Estructuras de datos aleatorizadas: treaps y skip lists

Las **estructuras de datos aleatorizadas** aprovechan la generación de números pseudoaleatorios para equilibrar o simplificar operaciones de inserción, eliminación y búsqueda. En lugar de mantener reglas de balanceo estrictas, estas estructuras obtienen, en promedio, un rendimiento logarítmico con gran simplicidad en su diseño.

**Treaps** combinan la propiedad de los árboles binarios de búsqueda (BST) con la de los heaps. Cada nodo posee una clave y una prioridad asignada de manera aleatoria. La clave determina la posición en el BST, mientras que la prioridad garantiza que el árbol mantenga, en promedio, un equilibrio, permitiendo operaciones en $O(\log n)$ en promedio. Su mecanismo de rotaciones al insertar o eliminar elementos simplifica el mantenimiento del balance sin necesidad de estructuras de balanceo complejas.

Una variante relacionada es el **randomized binary search tree (RBST)**. En este tipo de árbol, la posición de cada nodo se decide al azar durante la inserción, de manera que cada posible posición en el árbol es elegida con igual probabilidad. La inserción "aleatorizada" evita la degradación al peor caso, algo que podría ocurrir en un BST tradicional sin balanceo, y resulta en tiempos de operación esperados de $O(\log n)$.

Otra estructura interesante es el **randomized meldable heap**. A diferencia de los heaps tradicionales (como el binario, Fibonacci o el pairing heap), los meldable heaps no requieren una estructura rígida para realizar operaciones de fusión (merge). En la versión aleatorizada, al combinar dos heaps se decide al azar qué subárbol se unirá primero. Esta propiedad permite que la estructura sea especialmente útil cuando se requiere unir dos colas de prioridad, manteniendo un rendimiento promedio de $O(\log n)$ sin la complejidad de mantener invariantes fuertes de balanceo.

En el ámbito multidimensional, se pueden encontrar variantes como las **cartesian trees** o **randomized incremental structures**. Estas estructuras utilizan técnicas de inserción aleatoria para construir árboles que, por ejemplo, sirven como base para algoritmos de geometría computacional o para resolver problemas de rango de forma eficiente. Aunque su diseño puede diferir en función del problema particular, la idea central es la misma: aprovechar la aleatoriedad para lograr un equilibrio esperado y simplificar los algoritmos.


**¿Qué es una skip list?**

Una **skip list** es una estructura de datos probabilística que permite realizar operaciones de búsqueda, inserción y eliminación en promedio en tiempo $O(\log n)$. La clave de su funcionamiento es utilizar múltiples niveles de listas enlazadas; en cada nivel se "saltan" varios elementos, lo que permite acceder rápidamente a la posición deseada. La asignación de niveles se realiza de forma aleatoria, lo que simplifica tanto la implementación como el análisis teórico de su rendimiento.

En este cuaderno se implementan:
- La clase `Treap` y su nodo (`TreapNode`) para demostrar cómo la aleatoriedad ayuda a mantener un árbol balanceado.
- La clase `SkipList` y su nodo (`SkipListNode`), ilustrando otra forma de aprovechar el azar para obtener un rendimiento logarítmico en promedio.
- Funciones de demostración, pruebas unitarias mediante `unittest` y análisis de rendimiento utilizando `cProfile`.


In [ ]:
import random
import math
import time

# Clase para el nodo del treap

class TreapNode:
    """
    Nodo de un Treap.
    
    Atributos:
        key: Valor de la clave (usado para el orden en el BST).
        priority: Prioridad asignada aleatoriamente que respeta la propiedad heap.
        left: Hijo izquierdo.
        right: Hijo derecho.
    """
    def __init__(self, key):
        self.key = key
        self.priority = random.random()  # Prioridad aleatoria.
        self.left = None
        self.right = None

    def __str__(self):
        return f"(Clave: {self.key}, Prioridad: {self.priority:.4f})"


# Clase para la estructura treap

class Treap:
    """
    Implementación del treap con métodos de inserción, eliminación, búsqueda y recorrido.
    """
    def __init__(self):
        self.root = None

    def rotate_right(self, y):
        x = y.left
        t2 = x.right
        x.right = y
        y.left = t2
        return x

    def rotate_left(self, x):
        y = x.right
        t2 = y.left
        y.left = x
        x.right = t2
        return y

    def insert(self, root, key):
        if root is None:
            return TreapNode(key)
        if key < root.key:
            root.left = self.insert(root.left, key)
            if root.left.priority > root.priority:
                root = self.rotate_right(root)
        else:
            root.right = self.insert(root.right, key)
            if root.right.priority > root.priority:
                root = self.rotate_left(root)
        return root

    def insert_key(self, key):
        self.root = self.insert(self.root, key)

    def search(self, root, key):
        if root is None or root.key == key:
            return root
        if key < root.key:
            return self.search(root.left, key)
        return self.search(root.right, key)

    def search_key(self, key):
        return self.search(self.root, key)

    def delete(self, root, key):
        if root is None:
            return None
        if key < root.key:
            root.left = self.delete(root.left, key)
        elif key > root.key:
            root.right = self.delete(root.right, key)
        else:
            if root.left is None:
                return root.right
            elif root.right is None:
                return root.left
            else:
                if root.left.priority > root.right.priority:
                    root = self.rotate_right(root)
                    root.right = self.delete(root.right, key)
                else:
                    root = self.rotate_left(root)
                    root.left = self.delete(root.left, key)
        return root

    def delete_key(self, key):
        self.root = self.delete(self.root, key)

    def inorder(self, root):
        result = []
        if root:
            result.extend(self.inorder(root.left))
            result.append(root.key)
            result.extend(self.inorder(root.right))
        return result

    def display(self, root, indent=0):
        if root is not None:
            self.display(root.right, indent + 4)
            print(" " * indent + f"{root}")
            self.display(root.left, indent + 4)

# Clases para Skip List

class SkipListNode:
    """
    Nodo en una Skip List.
    
    Atributos:
        key: La clave o valor del nodo.
        forward: Lista de punteros a nodos en niveles múltiples.
    """
    def __init__(self, key, level):
        self.key = key
        self.forward = [None] * (level + 1)

    def __str__(self):
        return f"[{self.key}]"

class SkipList:
    """
    Implementación de la Skip List.
    
    Se utilizan niveles aleatorios para dar soporte a operaciones de búsqueda, inserción y eliminación en promedio O(log n).
    """
    def __init__(self, max_level, p):
        self.MAX_LEVEL = max_level
        self.p = p
        self.level = 0
        self.header = SkipListNode(-1, self.MAX_LEVEL)

    def random_level(self):
        lvl = 0
        while random.random() < self.p and lvl < self.MAX_LEVEL:
            lvl += 1
        return lvl

    def insert(self, key):
        update = [None] * (self.MAX_LEVEL + 1)
        current = self.header
        for i in range(self.level, -1, -1):
            while current.forward[i] and current.forward[i].key < key:
                current = current.forward[i]
            update[i] = current
        current = current.forward[0]
        if current is None or current.key != key:
            lvl = self.random_level()
            if lvl > self.level:
                for i in range(self.level + 1, lvl + 1):
                    update[i] = self.header
                self.level = lvl
            new_node = SkipListNode(key, lvl)
            for i in range(lvl + 1):
                new_node.forward[i] = update[i].forward[i]
                update[i].forward[i] = new_node

    def search(self, key):
        current = self.header
        for i in range(self.level, -1, -1):
            while current.forward[i] and current.forward[i].key < key:
                current = current.forward[i]
        current = current.forward[0]
        if current and current.key == key:
            return current
        return None

    def delete(self, key):
        update = [None] * (self.MAX_LEVEL + 1)
        current = self.header
        for i in range(self.level, -1, -1):
            while current.forward[i] and current.forward[i].key < key:
                current = current.forward[i]
            update[i] = current
        current = current.forward[0]
        if current and current.key == key:
            for i in range(self.level + 1):
                if update[i].forward[i] != current:
                    break
                update[i].forward[i] = current.forward[i]
            while self.level > 0 and self.header.forward[self.level] is None:
                self.level -= 1

    def display_list(self):
        print("\n Skip List")
        for i in range(self.level, -1, -1):
            current = self.header.forward[i]
            line = f"Nivel {i}: "
            while current:
                line += str(current.key) + " "
                current = current.forward[i]
            print(line)


**Funciones de demostración y bloque principal**

In [ ]:
# Funciones de demostración y análisis

def demo_treap():
    print("Demostración del treap")
    keys = [50, 30, 20, 40, 70, 60, 80]
    treap = Treap()
    for key in keys:
        treap.insert_key(key)
        print(f"Inserción de {key} realizada.")
    print("\nRecorrido inorder (claves ordenadas):")
    inorder_keys = treap.inorder(treap.root)
    print(inorder_keys)
    print("\nEstructura del treap (vista inclinada):")
    treap.display(treap.root)
    buscar = 40
    found = treap.search_key(buscar)
    if found:
        print(f"\nElemento {buscar} encontrado: {found}")
    else:
        print(f"\nElemento {buscar} no se encontró.")
    eliminar = 30
    print(f"\nEliminando {eliminar} del Treap...")
    treap.delete_key(eliminar)
    print("Nuevo recorrido inorder:")
    print(treap.inorder(treap.root))
    print("\nEstructura del treap tras eliminación:")
    treap.display(treap.root)
    print("="*40, "\n")

def demo_skip_list():
    print("Demostración de skip list")
    max_level = 4
    p = 0.5
    skip_list = SkipList(max_level, p)
    values = [3, 6, 7, 9, 12, 19, 17, 26, 21, 25]
    for value in values:
        skip_list.insert(value)
        print(f"Insertado {value} en Skip List.")
    skip_list.display_list()
    buscar_val = 19
    result = skip_list.search(buscar_val)
    if result:
        print(f"\nElemento {buscar_val} encontrado en skip list.")
    else:
        print(f"\nElemento {buscar_val} no encontrado en skip list.")
    eliminar_val = 7
    print(f"\nEliminando {eliminar_val} de la skip list...")
    skip_list.delete(eliminar_val)
    print("Skip list después de eliminar:")
    skip_list.display_list()
    print("="*40, "\n")

def performance_analysis():
    print("Analisis de rendimiento")
    n = 1000  # Número de elementos a insertar
    keys = random.sample(range(1, 10000), n)
    # Análisis para el treap.
    treap = Treap()
    inicio = time.time()
    for key in keys:
        treap.insert_key(key)
    fin = time.time()
    recorrido = treap.inorder(treap.root)
    print(f"Treap: Se insertaron {n} elementos en {fin - inicio:.4f} segundos.")
    print(f"Profundidad aproximada (cantidad de claves en recorrido inorder): {len(recorrido)}")
    # Análisis para la Skip List.
    max_level = int(math.log(n, 2)) if n > 0 else 1
    skip_list = SkipList(max_level, 0.5)
    inicio = time.time()
    for key in keys:
        skip_list.insert(key)
    fin = time.time()
    print(f"\nSkip List: Se insertaron {n} elementos en {fin - inicio:.4f} segundos.")
    print("="*40, "\n")

# Bloque principal
demo_treap()
time.sleep(1)
demo_skip_list()
performance_analysis()

# Ejemplo adicional: aplicación en algoritmos probabilísticos.
print("Ejemplo de aplicación en algoritmos probabilisticos")
treap_app = Treap()
for _ in range(20):
    key = random.randint(1, 100)
    treap_app.insert_key(key)
print("Treap para aplicación probabilística:")
treap_app.display(treap_app.root)

skip_list_app = SkipList(5, 0.5)
for _ in range(20):
    key = random.randint(1, 100)
    skip_list_app.insert(key)
skip_list_app.display_list()

print("\nObserva cómo ambas estructuras permiten decisiones basadas en probabilidades, ayudando a evitar peores casos determinísticos.")


**Pruebas unitarias con unittest**

In [ ]:
import unittest

class TestEstructurasAleatorizadas(unittest.TestCase):
    # Pruebas para Treap
    def test_treap_insert_and_search(self):
        treap = Treap()
        keys = [15, 10, 20, 8, 12, 16, 25]
        for key in keys:
            treap.insert_key(key)
        # Verificar que cada clave insertada se pueda encontrar.
        for key in keys:
            node = treap.search_key(key)
            self.assertIsNotNone(node, f"La clave {key} debería existir en el Treap.")
            self.assertEqual(node.key, key)
        # Buscar una clave que no exista.
        self.assertIsNone(treap.search_key(99))
    
    def test_treap_delete(self):
        treap = Treap()
        keys = [15, 10, 20, 8, 12, 16, 25]
        for key in keys:
            treap.insert_key(key)
        treap.delete_key(10)
        self.assertIsNone(treap.search_key(10))
        inorder_keys = treap.inorder(treap.root)
        self.assertNotIn(10, inorder_keys)
    
    # Pruebas para Skip List
    def test_skip_list_insert_and_search(self):
        max_level = 4
        p = 0.5
        skip_list = SkipList(max_level, p)
        values = [5, 1, 7, 3, 9]
        for val in values:
            skip_list.insert(val)
        for val in values:
            node = skip_list.search(val)
            self.assertIsNotNone(node, f"La clave {val} debería existir en la Skip List.")
            self.assertEqual(node.key, val)
        self.assertIsNone(skip_list.search(100))
    
    def test_skip_list_delete(self):
        max_level = 4
        p = 0.5
        skip_list = SkipList(max_level, p)
        values = [5, 1, 7, 3, 9]
        for val in values:
            skip_list.insert(val)
        skip_list.delete(7)
        self.assertIsNone(skip_list.search(7))


## Ejecutando pruebas
unittest.main(argv=[''], verbosity=2, exit=False)


**Profiling con cProfile**

In [ ]:
import cProfile
import pstats
from io import StringIO

def run_profiling():
    pr = cProfile.Profile()
    pr.enable()
    
    # Se ejecutan las funciones de demostración
    demo_treap()
    demo_skip_list()
    performance_analysis()
    
    pr.disable()
    s = StringIO()
    sortby = 'cumulative'
    ps = pstats.Stats(pr, stream=s).sort_stats(sortby)
    ps.print_stats(10)  # Imprime las 10 funciones más costosas.
    print(s.getvalue())


run_profiling()

### **Ejercicios**

#### **1: Extensión de treap para soportar operaciones de split, merge y selección por orden**

Extender la implementación del `Treap` para que soporte operaciones adicionales muy utilizadas en algoritmos avanzados, como:

- **Split:** Dividir un treap en dos, uno con todos los elementos menores o iguales a una clave dada y otro con los elementos mayores.
- **Merge:** Unir dos treaps siempre que todas las claves del primer árbol sean menores que todas las claves del segundo.
- **Orden-statistics:** Implementar una función para obtener el *k*-ésimo elemento (por ejemplo, la mediana o cualquier otro percentil).

**Tareas:**

1. **Implementación:**
   - Agrega funciones `split(treap, key)` y `merge(treap1, treap2)` a la clase `Treap`.
   - Para la selección por orden, considera mantener información adicional (por ejemplo, el tamaño del subárbol) en cada nodo, lo que permita acceder en tiempo O(log n) al *k*-ésimo elemento.
  
2. **Pruebas:**
   - Usa `unittest` para crear casos de prueba que verifiquen la corrección de las operaciones `split` y `merge` en diferentes escenarios, incluyendo:
     - Árboles vacíos.
     - Árboles con un solo elemento.
     - Árboles grandes con inserciones aleatorias.
   - Verifica la correcta actualización de la información de tamaño tras cada operación.

3. **Profiling:**
   - Utiliza `cProfile` y el módulo `timeit` para medir el rendimiento de estas operaciones en conjuntos de datos de gran tamaño (por ejemplo, 10.000 o 100.000 elementos).
   - Analiza los resultados y discute si el comportamiento se aproxima a la complejidad teórica O(log n) para cada operación.

#### **2:Skip List con soporte para claves duplicadas y búsqueda por rango**
 Extender la implementación actual de Skip List para:
  
- Permitir la inserción de claves duplicadas.
- Implementar una operación de búsqueda por rango, `range_query(low, high)`, que devuelva todos los elementos entre dos valores dados.

**Tareas:**

1. **Implementación:**
   - Modifica la estructura para que acepte duplicados. Esto puede implicar almacenar una lista o contador en cada nodo para las repeticiones.
   - Implementa `range_query(low, high)` que recorra los niveles inferiores de la `Skip List` y devuelva todas las claves en el intervalo `[low, high]`.

2. **Pruebas:**
   - Diseña pruebas unitarias que verifiquen la inserción correcta de duplicados y la validez del resultado del método `range_query`.
   - Crea escenarios de prueba con valores límite y cantidades elevadas de datos para asegurarte de que la implementación es robusta.

3. **Profiling:**
   - Realiza análisis de rendimiento usando `cProfile` en operaciones de inserción, búsqueda y consulta por rango en conjuntos de datos de diferentes tamaños.
   - Identifica posibles cuellos de botella y discute estrategias de optimización en el caso de búsquedas por rangos muy amplios.

#### **3: Análisis comparativo y visualización del rendimiento en escenarios reales**

Comparar el rendimiento de `Treap` y `Skip List` en distintos escenarios de uso real, donde los patrones de inserción y consulta sean variados.

**Tareas:**

1. **Diseño de experimentos:**
   - Crea un conjunto de experimentos que contemple:
     - Inserciones secuenciales (orden ascendente y descendente).
     - Inserciones aleatorias.
     - Búsquedas intensivas después de inserciones masivas.
     - Eliminación masiva de elementos.
   - Para cada escenario, genera múltiples repeticiones para obtener tiempos promedio fiables.

2. **Implementación y perfilado:**
   - Utiliza `cProfile` y `timeit` para capturar los tiempos de ejecución de las operaciones críticas en cada estructura.
   - Almacena los resultados en estructuras que te permitan graficar (por ejemplo, usando `matplotlib`) la comparación entre ambas estructuras.

3. **Visualización y análisis:**
   - Genera gráficos que muestren cómo varía el rendimiento de cada estructura según el escenario.
   - Realiza un análisis crítico en el que discutas cuál estructura es más adecuada en cada caso y por qué, considerando tanto tiempo de procesamiento como uso de memoria.


#### **4: Testing y profiling en contextos concurrentes**

Explorar el comportamiento de la implementación en ambientes concurrentes, incluso cuando las estructuras no fueron diseñadas originalmente para ello.

**Tareas:**

1. **Implementación:**
   - Modifica las clases `Treap` y `SkipList` para agregar mecanismos de sincronización (por ejemplo, utilizando `threading.Lock` o `RLock`) para que sean seguras ante concurrencia.
   - Permite que múltiples hilos puedan realizar operaciones de inserción, búsqueda y eliminación sin corromper la estructura.

2. **Testing concurrente:**
   - Desarrolla pruebas unitarias utilizando `concurrent.futures.ThreadPoolExecutor` para simular acceso concurrente.
   - Verifica la integridad de la estructura tras ejecuciones en paralelo y asegura que no se produzcan condiciones de carrera.

3. **Profiling:**
   - Analiza el impacto del locking sobre el rendimiento utilizando `cProfile`.
   - Compara el rendimiento y la latencia en escenarios concurrentes frente a escenarios secuenciales, y discute las implicaciones de escalar el acceso en entornos multihilo.


#### **5: Análisis detallado del uso de memoria y optimización**

Utilizar herramientas de análisis de memoria (por ejemplo, `tracemalloc`) para identificar y optimizar puntos críticos en la asignación de memoria en ambas estructuras.

**Tareas:**

1. **Implementación:**
   - Integra `tracemalloc` en el script para capturar el uso de memoria antes y después de operaciones masivas de inserción y eliminación.
   - Implementa funciones que registren y reporten el pico de memoria utilizado durante la ejecución de tareas críticas.

2. **Testing:**
   - Diseña escenarios donde se inserten, modifiquen y eliminen grandes cantidades de datos, y captura un informe detallado de la memoria.
   - Compara la huella de memoria entre Treap y Skip List.

3. **Profiling y optimización:**
   - Utiliza los reportes de `tracemalloc` en conjunto con los análisis de CPU de `cProfile` para identificar funciones o secciones del código que consuman recursos excesivos.
   - Basándote en los resultados, realiza optimizaciones (p.ej., evitando estructuras intermedias innecesarias, mejorando la asignación dinámica, etc.).
   - Documenta los cambios realizados y cómo éstos impactan en la eficiencia general.


In [ ]:
## Tus respuestas